In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
from pyspark.sql.functions import *

filepath='file:////home/talentum/shared/Project/Project_codes/section5/15stocks_price.csv'
df = spark.read.option("header", True).csv(filepath)
df = df.withColumn("date_gsheets",to_date(df.date_gsheets,"MM/dd/yyyy HH:mm"))
df = df.withColumnRenamed('BRK.B', 'BRK')
df.show()

+------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+
|date_gsheets| AAPL|   KO| NFLX|  BRK|  DIS|   IBM|   VZ|  WMT|   GE| TSLA|   MA|  AMZN| MSFT|   UN|    V|
+------------+-----+-----+-----+-----+-----+------+-----+-----+-----+-----+-----+------+-----+-----+-----+
|  2012-01-03|58.75|35.07|10.32|77.68|38.31| 186.3|39.73|60.33|18.36|28.08|36.84|179.03|26.77|34.92|25.75|
|  2012-01-04|59.06|34.85|11.49| 76.8|38.85|185.54|39.21|59.71|18.56|27.71|35.63|177.51| 27.4|34.45|25.29|
|  2012-01-05|59.72|34.69|11.33|76.93| 39.5|184.66|38.94|59.42|18.55|27.12|35.24|177.61|27.68|34.49|25.48|
|  2012-01-06|60.34|34.47|12.33|76.39|39.91|182.54|38.33|   59|18.65|26.91|34.29|182.61|28.11|33.27|25.18|
|  2012-01-09|60.25|34.47|14.03|76.29|39.75|181.59|38.37|59.18|18.86|27.25|34.58|178.56|27.74|33.72|24.98|
|  2012-01-10|60.46|34.67|13.69|77.42|39.63|181.31|38.57|59.04|18.72|27.62|34.88|179.34|27.84|33.91|24.95|
|  2012-01-11|60.36|34.03|13.16|   78

In [4]:
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import functions as F

return_df=df
my_window = Window.partitionBy().orderBy("date_gsheets")
for col_name in return_df.columns:
    if col_name!='date_gsheets':
     return_df = return_df.withColumn(col_name, F.when(F.isnull(col(col_name) - F.lag(col(col_name)).over(my_window)), 0)
                                      .otherwise(col(col_name) - F.lag(col(col_name)).over(my_window))/F.lag(col(col_name)).over(my_window))
for c in return_df.columns:
    if c!='date_gsheets':
        return_df = return_df.withColumn(c, F.round(c, 6))

return_df=return_df.dropna()
return_df=return_df.drop('date_gsheets')
return_df.show()

+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|     AAPL|       KO|     NFLX|      BRK|      DIS|      IBM|       VZ|      WMT|       GE|     TSLA|       MA|     AMZN|     MSFT|       UN|        V|
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| 0.005277|-0.006273| 0.113372|-0.011329| 0.014096|-0.004079|-0.013088|-0.010277| 0.010893|-0.013177|-0.032845| -0.00849| 0.023534|-0.013459|-0.017864|
| 0.011175|-0.004591|-0.013925| 0.001693| 0.016731|-0.004743|-0.006886|-0.004857| -5.39E-4|-0.021292|-0.010946|  5.63E-4| 0.010219| 0.001161| 0.007513|
| 0.010382|-0.006342| 0.088261|-0.007019|  0.01038|-0.011481|-0.015665|-0.007068| 0.005391|-0.007743|-0.026958| 0.028152| 0.015535|-0.035373|-0.011774|
|-0.001492|      0.0| 0.137875|-0.001309|-0.004009|-0.005204| 0.001044| 0.003051|  0.011

In [5]:
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import DenseMatrix
from pyspark.sql.functions import sqrt

def getPortRisk(weights):
    
    '''Returns the annualised standard deviation of a k asset portfolio.'''

    return_rdd = return_df.rdd.map(list)
    return_row_mat = RowMatrix(return_rdd)
    vcv_matrix=return_row_mat.computeCovariance()
        
    weight_matrix= DenseMatrix(1, num_stocks, weights)
    weight_matrix_t= DenseMatrix(num_stocks, 1, weights)
    weight_row_mat = RowMatrix(sc.parallelize(weight_matrix.toArray().tolist()))
    vcv_w=weight_row_mat.multiply(vcv_matrix)
    var_portfolio = vcv_w.multiply(weight_matrix_t).rows.collect()

    var_portfolio=var_portfolio[0][0]
    sd_portfolio=np.sqrt(var_portfolio)
    sd_portfolio_annual = sd_portfolio * np.sqrt(250)
           
    return sd_portfolio_annual

In [6]:
num_stocks = len(return_df.columns)  # being the number of stocks (this is a 'global' variable)
init_weights = [1 / num_stocks] * num_stocks  # initialise weights (x0)

In [7]:
# Constraint that weights in any asset j must be between 0 and 1 inclusive
bounds = tuple((0, 1) for i in range(num_stocks))

In [8]:
# Constraint that the sum of the weights of all assets must equate to 1
cons = ({'type' : 'eq', 'fun' : lambda x : np.sum(x) - 1})

In [9]:
from scipy.optimize import minimize
import numpy as np
results = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons)

In [10]:
results

     fun: 0.10593481497482597
     jac: array([0.10597393, 0.10585238, 0.10600165, 0.10580072, 0.10597587,
       0.10618787, 0.1061356 , 0.10582462, 0.105877  , 0.11916326,
       0.11288884, 0.10579357, 0.1093439 , 0.10589382, 0.10634916])
 message: 'Optimization terminated successfully'
    nfev: 176
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([4.79793405e-02, 2.80743080e-01, 7.53117077e-03, 1.29295717e-01,
       3.54257541e-02, 8.40451351e-02, 1.62830301e-01, 1.72867978e-01,
       2.00871198e-02, 1.78893358e-18, 1.57209315e-18, 3.55689476e-03,
       5.42101086e-20, 4.31646087e-02, 1.24729006e-02])

In [13]:
getPortRisk(init_weights)

0.13290601472607508

In [17]:
rdd=sc.parallelize(results['x'])
optimised_weights=spark.createDataFrame(rdd.map(lambda x: float(x)), FloatType())
optimised_weights=optimised_weights.withColumnRenamed("value","Optimized_Weights")
optimised_weights.show()

+-----------------+
|Optimized_Weights|
+-----------------+
|       0.04797934|
|        0.2807431|
|     0.0075311707|
|       0.12929572|
|      0.035425752|
|      0.084045134|
|       0.16283031|
|       0.17286798|
|       0.02008712|
|    1.7889336E-18|
|    1.5720932E-18|
|     0.0035568948|
|     5.421011E-20|
|      0.043164607|
|      0.012472901|
+-----------------+



In [26]:
# Create a dataframe of all the portfolio risks together with protfolio columns

stocks_df = spark.createDataFrame([(l,) for l in return_df.columns], ["Stocks"])

# Add sequential index to both DataFrames
optimised_weights= optimised_weights.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))
stocks_df = stocks_df.withColumn("row_idx", row_number().over(Window.orderBy(monotonically_increasing_id())))

# Join DataFrames on the index and drop the index column
optimised_weights_df = stocks_df.join(optimised_weights, "row_idx").drop("row_idx")

optimised_weights_df = optimised_weights_df.withColumn("Optimized_Weights", round(optimised_weights_df["Optimized_Weights"], 3))

# Show the result
optimised_weights_df.show()

+------+-----------------+
|Stocks|Optimized_Weights|
+------+-----------------+
|  AAPL|            0.048|
|    KO|            0.281|
|  NFLX|            0.008|
|   BRK|            0.129|
|   DIS|            0.035|
|   IBM|            0.084|
|    VZ|            0.163|
|   WMT|            0.173|
|    GE|             0.02|
|  TSLA|              0.0|
|    MA|              0.0|
|  AMZN|            0.004|
|  MSFT|              0.0|
|    UN|            0.043|
|     V|            0.012|
+------+-----------------+



In [28]:
from pyspark.sql.functions import col, sum
optimised_weights_cumsum = optimised_weights_df.withColumn("cumsum", sum(col("Optimized_Weights"))
                                 .over(Window.orderBy(col("Optimized_Weights").desc())))
optimised_weights_cumsum = optimised_weights_cumsum.withColumn("cumsum", round(optimised_weights_cumsum["cumsum"], 3))
optimised_weights_cumsum.show()

+------+-----------------+------+
|Stocks|Optimized_Weights|cumsum|
+------+-----------------+------+
|    KO|            0.281| 0.281|
|   WMT|            0.173| 0.454|
|    VZ|            0.163| 0.617|
|   BRK|            0.129| 0.746|
|   IBM|            0.084|  0.83|
|  AAPL|            0.048| 0.878|
|    UN|            0.043| 0.921|
|   DIS|            0.035| 0.956|
|    GE|             0.02| 0.976|
|     V|            0.012| 0.988|
|  NFLX|            0.008| 0.996|
|  AMZN|            0.004|   1.0|
|  TSLA|              0.0|   1.0|
|    MA|              0.0|   1.0|
|  MSFT|              0.0|   1.0|
+------+-----------------+------+

